## Single dataloader

Try and make a torch dataloader containing just one map from camels

In [28]:
import numpy as np
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import time, sys, os
import matplotlib.pyplot as plt

# my modules
from sn_camels.models.models_factory import baseModelFactory, topModelFactory
from sn_camels.models.sn_hybrid_models import sn_HybridModel
from sn_camels.camels.camels_dataset import *
from sn_camels.models.camels_models import model_o3_err

In [2]:
## camels path
camels_path=os.environ['CAMELS_PATH']

# data parameters
fmaps      = ['maps_Mcdm.npy'] #tuple containing the maps with the different fields to consider
fparams    = camels_path+"/params_IllustrisTNG.txt"

In [56]:
class single_map():

    def __init__(self,f_images, f_params):

        data   = np.load(f_images)    #[number of maps, height, width]
        params = np.loadtxt(f_params) #parameters of simulations, not of maps
    
        self.x=torch.tensor([data[0]])
        self.y=torch.tensor([params[0]])
        print(self.y)
        self.size=1
    
    
    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [57]:
test_single=single_map(fmaps[0],fparams)

tensor([[0.3090, 0.9790, 3.1123, 1.1219, 0.6685, 0.5318]], dtype=torch.float64)


/tmp/ipykernel_47686/2725099591.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  self.x=torch.tensor([data[0]])


In [58]:
test_single.__len__()

1

In [59]:
test_single.x.shape

torch.Size([1, 256, 256])

In [60]:
data_loader = DataLoader(dataset=test_single, batch_size=1, shuffle=False,
                             num_workers=1)

In [61]:
model = model_o3_err(3, 0.2, 1)

In [62]:
device = torch.device('cpu')
# training parameters
channels        = 1                #we only consider here 1 field
params          = [0,1,2,3,4,5]    #0(Omega_m) 1(sigma_8) 2(A_SN1) 3 (A_AGN1) 4(A_SN2) 5(A_AGN2). The code will be trained to predict all these parameters.
g               = params           #g will contain the mean of the posterior
h               = [6+i for i in g] #h will contain the variance of the posterior

In [63]:
x, y=next(iter(data_loader))

In [72]:
model(x)

IndexError: tuple index out of range

In [67]:
for epoch in range(5):

    # do training
    train_loss1, train_loss2 = torch.zeros(len(g)).to(device), torch.zeros(len(g)).to(device)
    train_loss, points = 0.0, 0
    model.train()
    bs   = x.shape[0]         #batch size
    x    = x.to(device)       #maps
    y    = y.to(device)[:,g]  #parameters
    p    = model(x)           #NN output
    y_NN = p[:,g]             #posterior mean
    e_NN = p[:,h]             #posterior std
    loss1 = torch.mean((y_NN - y)**2,                axis=0)
    loss2 = torch.mean(((y_NN - y)**2 - e_NN**2)**2, axis=0)
    loss  = torch.mean(torch.log(loss1) + torch.log(loss2))
    train_loss1 += loss1*bs
    train_loss2 += loss2*bs
    points      += bs
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

        #if points>18000:  break
    train_loss = torch.log(train_loss1/points) + torch.log(train_loss2/points)
    train_loss = torch.mean(train_loss).item()

IndexError: tuple index out of range